In [ ]:
#hide
#default_exp update_project
from nbdev.showdoc import show_doc

In [ ]:
#export
import os
import pkg_resources

import jinja2
from fastcore.script import *

from kicad_helpers import get_git_root, get_project_metadata, get_gitignore_list, in_gitignore

# update_project

> Functions scripts to initialize a new kicad-helpers project or to update an existing project.

* toc: true

Similar to `nbdev`: [nbdev_new](https://nbdev.fast.ai/tutorial.html#Set-up-Repo)

* [x] github workflows
* [x] .kicad_helpers_config
* [x] kitspace.yaml
* [ ] Example notebooks (e.g., for syncing the BOM)

Templates are stored in the "kicad_helpers/templates" folder, and are included in the python package by adding the following line to the `MANIFEST.in` file:

```
graft kicad_helpers/templates
```

In [ ]:
#export
@call_parse
def update_templates(root=None):
    # Render all files in the `templates` directory (ignoring anything that is
    # in the `.gitignore` list) as jinja2 templates.
    templates_path = os.path.abspath(pkg_resources.resource_filename('kicad_helpers', 'templates'))
    
    if root is None:
        # Use defaults
        root = get_git_root(".")
        
        # Override with environment variable if set
        root = os.getenv("KH_PROJECT_ROOT", root)
    
    metadata = get_project_metadata(root)
    
    file_list = []
    for root_, dirs, files in os.walk(templates_path):
        if len(files):
            for file in files:
                path = os.path.join(root_[len(templates_path) + 1:], file)
                if not in_gitignore(path):
                    src_path = os.path.abspath(os.path.join(templates_path, path))
                    dst_path = os.path.abspath(os.path.join(root, path))
                    
                    # Create the `dst_path` directory if it doesn't exist
                    os.makedirs(os.path.split(dst_path)[0], exist_ok=True)

                    if os.path.exists(dst_path):
                        print(f"{dst_path} already exists")
    
                    with open(src_path) as f:
                        template = jinja2.Template(f.read())

                    with open(dst_path, "w") as f:
                        f.write(template.render(**metadata))

In [ ]:
import tempfile

from kicad_helpers import setup_test_repo

temp_dir = tempfile.mkdtemp()
setup_test_repo(temp_dir)
update_templates(temp_dir)
os.chdir(temp_dir)

Cloning into '/tmp/tmpc8ijn9js'...


/tmp/tmpc8ijn9js/kitspace.yaml already exists
/tmp/tmpc8ijn9js/.github/workflows/build.yml already exists
/tmp/tmpc8ijn9js/.kicad_helpers_config/config.kibot.yaml already exists
/tmp/tmpc8ijn9js/.kicad_helpers_config/manufacturers/PCBWay.kibot.yaml already exists


# Project structure

Here's an example showing the recommended KiCad project structure.

In [ ]:
%%sh
tree --dirsfirst -d -I $(python -c "from kicad_helpers import get_gitignore_list; print('kicad_helpers|' + get_gitignore_list())") .

.
├── docs
│   ├── png
│   └── src
├── library
└── manufacturing
    └── default
        ├── assembly_photos
        ├── gerbers
        └── position

9 directories
